In [2]:
import logging
import os
import json
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv
import time
import requests
from tenacity import retry, stop_after_attempt, wait_fixed
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import functools
import mistral_functions_list
from actions.content_creator.college_board_api import get_institution
import json

In [3]:
load_dotenv()

# Mistral API key and configuration
MISTRAL_API_KEY = os.environ["MISTRAL_API_KEY"]
MODEL_NAME = "mistral-large-latest"
monthly_token_limit = 10000000000

current_monthly_usage = 0
client = MistralClient(api_key=MISTRAL_API_KEY)

In [11]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_institution",
            "description": "Get school information from the College Scorecard.",
            "parameters": {
                "type": "object",
                "properties": {
                    "name": {
                        "type": "string",
                        "description": "The name of the school to search for.",
                    },
                    "zip": {
                        "type": "string",
                        "description": "Postal code of the school.",
                    },
                    "city": {
                        "type": "string",
                        "description": "City of the school.",
                    },
                    "state": {
                        "type": "string",
                        "description": "State of the school.",
                    },
                },
            },
        },
    },
]

In [12]:

messages = [
    ChatMessage(
        role="user",
        content="Retrive all information about the school",
        tools=tools,
        tool_choice="auto",
    )
]

response = client.chat(
    model=MODEL_NAME, messages=messages, tools=tools, tool_choice="auto"
)
response

ChatCompletionResponse(id='cfa3a338890947d78c0716668e3021ff', object='chat.completion', created=1712561439, model='mistral-large-latest', choices=[ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content='To retrieve information about a school, I would need some specific details such as the name of the school, city, state, or zip code. Could you please provide these details?', name=None, tool_calls=None), finish_reason=<FinishReason.stop: 'stop'>)], usage=UsageInfo(prompt_tokens=140, total_tokens=176, completion_tokens=36))

In [13]:
messages.append(ChatMessage(role="assistant", content=response.choices[0].message.content))
messages.append(ChatMessage(role="user", content="The school name is University of California-Berkeley"))
response = client.chat(model=MODEL_NAME, messages=messages, tools=tools, tool_choice="auto")
response

ChatCompletionResponse(id='4fe2269775d24e69ab50123ced6465ee', object='chat.completion', created=1712561446, model='mistral-large-latest', choices=[ChatCompletionResponseChoice(index=0, message=ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='null', type=<ToolType.function: 'function'>, function=FunctionCall(name='get_institution', arguments='{"name": "University of California-Berkeley"}'))]), finish_reason=<FinishReason.tool_calls: 'tool_calls'>)], usage=UsageInfo(prompt_tokens=189, total_tokens=217, completion_tokens=28))

In [14]:
messages.append(response.choices[0].message)
messages

[ChatMessage(role='user', content='Retrive all information about the school', name=None, tool_calls=None),
 ChatMessage(role='assistant', content='To retrieve information about a school, I would need some specific details such as the name of the school, city, state, or zip code. Could you please provide these details?', name=None, tool_calls=None),
 ChatMessage(role='user', content='The school name is University of California-Berkeley', name=None, tool_calls=None),
 ChatMessage(role='assistant', content='', name=None, tool_calls=[ToolCall(id='null', type=<ToolType.function: 'function'>, function=FunctionCall(name='get_institution', arguments='{"name": "University of California-Berkeley"}'))])]

In [15]:
tool_call = response.choices[0].message.tool_calls[0]
function_name = tool_call.function.name
function_params = json.loads(tool_call.function.arguments)
print("\nfunction_name: ", function_name, "\nfunction_params: ", function_params)


function_name:  get_institution 
function_params:  {'name': 'University of California-Berkeley'}


In [20]:
names_to_functions = {
    "get_institution": functools.partial(
        get_institution
    )
}
function_result = await names_to_functions["get_institution"](**function_params)

if isinstance(function_result, (list, dict)):
    function_result = json.dumps(function_result)
print(function_result)
# print(f"Total token count: {len(function_result)}")


HTTPException: 404: School not found

In [16]:
messages.append(ChatMessage(role="tool", name=function_name, content=function_result))

response = client.chat(model=MODEL_NAME, messages=messages)
response.choices[0].message.content

NameError: name 'function_result' is not defined